# Module 6

## Video 26: Inspecting Cargo Movements
**Python for the Energy Industry**

In this lesson, we take a closer look at the structure of a Cargo Movement. 

[Cargo movements documentation.](https://vortechsa.github.io/python-sdk/endpoints/cargo_movements/)

In [1]:
# imports
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import vortexasdk as v
now = datetime.utcnow()

Let's start with a basic query to the SDK, for all Cargo Movements that are currently loading. We then look at one of these:

In [2]:
# basic query
cm_query = v.CargoMovements().search(
    filter_activity="loading_state",
    filter_time_min=now,
    filter_time_max=now)

# remember - cm_query is a *list* of cargo movements

# taking a look at a single Cargo Movement
cm_query[0]

2020-12-10 17:17:54,405 vortexasdk.client — WARNING — You are using vortexasdk version 0.26.0, however version 0.27.1 is available.
You should consider upgrading via the 'pip install vortexasdk --upgrade' command.


Loading from API: 1000it [00:01, 888.43it/s]             


{'cargo_movement_id': 'a50ce7fdb701c258b74ad260bf946146a50ce7fdb701c258b74ad260bf946146',
 'quantity': 154106,
 'status': 'unloaded_state',
 'vessels': [{'id': 'd7a6983fded96c41c2118ca08a3a29b6c601a3e29b4ceb8b5bafe529345de59d',
   'mmsi': 538007372,
   'imo': 9364588,
   'name': 'JANE',
   'dwt': 51506,
   'cubic_capacity': 53511,
   'vessel_class': 'handymax',
   'corporate_entities': [{'id': '40692347e32537124e6458690fcff72bc7deb01606409e9bc9ad198ed003ad32',
     'label': 'Product Shipping & Trading',
     'layer': 'commercial_owner',
     'probability': 1,
     'source': 'external'}],
   'start_timestamp': '2020-10-31T12:05:29+0000',
   'end_timestamp': '2020-11-14T22:31:32+0000',
   'fixture_fulfilled': False,
   'voyage_id': '524edfdabdeb742d06589ce41cfe434db1ce1c6f7ae23cd2a0eee11387c2e745',
   'tags': [{'tag': 'vessel_coated_tag'}],
   'status': 'vessel_status_laden_unknown',
   'year': 2008,
   'scrubber': [],
   'flag': [{'tag': 'vessel_flag_tag',
     'flag': 'MH',
     'flag_

There's a lot of information here. Note that this is a 'dictionary' structure, so we can put out the top level keys, and their type:

In [3]:
cm_query[0].keys()

dict_keys(['cargo_movement_id', 'quantity', 'status', 'vessels', 'product', 'events', 'parent_ids'])

In [4]:
print([type(cm_query[0][cmk]) for cmk in cm_query[0]])

[<class 'str'>, <class 'int'>, <class 'str'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>]


Three of these keys correspond to individual vales which we can print out:

In [5]:
print('cargo_movement_id:', cm_query[0]['cargo_movement_id'])
print('quantity:', cm_query[0]['quantity'])
print('status:', cm_query[0]['status'])

cargo_movement_id: a50ce7fdb701c258b74ad260bf946146a50ce7fdb701c258b74ad260bf946146
quantity: 154106
status: unloaded_state


The remaning keys are lists, which we now now describe in turn.

`vessels` contains a `VesselEntity` for each vessel involved in the Cargo Movement. In general, there can be multiple vessels, but we shall touch on this later. The Cargo Movement we're looking only contains one `VesselEntity`:

In [8]:
vessels = cm_query[1]['vessels']
len(vessels)

1

In [9]:
vessels[0]

{'id': 'c66166bc9f07fa8c88dc6ea248afd3597cc758f65497f7b220ca54d00921c38c',
 'mmsi': 258682000,
 'imo': 9321689,
 'name': 'SOLVIKEN',
 'dwt': 114523,
 'cubic_capacity': 127731,
 'vessel_class': 'aframax',
 'corporate_entities': [{'id': '409dd25aeff466583e8d738f2972ab7d6032622b07c2722f6459edd1be32e1de',
   'label': 'BP',
   'layer': 'charterer',
   'probability': 1,
   'source': 'external'},
  {'id': '19b1bf392f474ca942b7b25ddcba363638516b2e40268ea7a70888021575a4dd',
   'label': 'VITOL',
   'layer': 'commercial_owner',
   'probability': 1,
   'source': 'external'}],
 'start_timestamp': '2020-10-31T08:46:01+0000',
 'end_timestamp': '2020-11-27T22:05:13+0000',
 'fixture_id': 'e572fa21bb5b48d4b1bdc956bc008cb50b919502e47ecc9aaab2705f23367b40',
 'fixture_fulfilled': False,
 'voyage_id': '6a05f4e387884086072fc56ab4736e1402b3acc6f3f7a69b0381153e359166aa',
 'tags': [],
 'status': 'vessel_status_laden_known',
 'year': 2007,
 'scrubber': [],
 'flag': [{'tag': 'vessel_flag_tag',
   'flag': 'NO',
  

`products` contains Product Entries, which each describe one layer of the product tree (Group, Group Product, Category, and Grade). Not all products specify a Grade. We can view the Product Entries in a DataFrame:

In [10]:
products = cm_query[0]['product']
pd.DataFrame(products)

,id,layer,probability,source,label
0,b68cbb746f8b9098c50e2ba36bcad83001a53bd362e903...,group,0.602556,model,Clean Petroleum Products
1,9256907ba7e4ed11ff03aa297a7e62e14484ce5a85c811...,group_product,0.602556,model,Gasoline/Blending Components
2,c76d43ed46524d628a16be1b63019719631a5c2dcbf54a...,category,0.602556,model,Finished Gasoline


`events` contains Cargo Events, which each describe a specific event happening to the cargo at a specific time. These can be:
- cargo_port_load_event
- cargo_port_unload_event
- cargo_fso_load_event
- cargo_fso_unload_event
- cargo_sts_event
- cargo_fixture_event
- cargo_storage_event

In our example, the two events are:

In [12]:
events = cm_query[1]['events']
[e['event_type'] for e in events]

['cargo_port_load_event', 'cargo_port_unload_event']

We can see some high level information about these events by putting them into a DataFrame:

In [13]:
pd.DataFrame(events)

,event_type,location,probability,pos,start_timestamp,end_timestamp
0,cargo_port_load_event,[{'id': '2d92cc08f22524dba59f6a7e340f132a9da0c...,1,"[-95.12550820901582, 29.73760530405836]",2020-10-31T08:46:01+0000,2020-11-01T19:10:20+0000
1,cargo_port_unload_event,[{'id': 'ee1de4914cc26e8f1326b49793b089131870d...,1,"[13.777823781603512, 45.610871763346225]",2020-11-26T09:43:47+0000,2020-11-27T22:05:13+0000


The 'location' entries are themselves dictionaries, which we can expand into a DataFrame to see the different layers:

In [14]:
pd.DataFrame(events[0]['location'])

,id,layer,label,source,probability
0,2d92cc08f22524dba59f6a7e340f132a9da0ce9573cca9...,country,United States,model,1
1,7f314ba0a498c36359b1c88781e94a73e19dcc9bbb030e...,port,"Houston, TX [US]",model,1
2,ae06d7e47d6ca2a5bcd7625b8ad402d39e5e92fcc74015...,region,North America,model,1
3,3f0a65d97728cdd1aaa5830d2de018d4378f2c453a89bc...,shipping_region,Gulf of Mexico,model,1
4,4e79eb8e84d26f5c3c0006283bc1aa52c170b58d667be9...,shipping_region,PADD 3 (US Gulf Coast),model,1
5,f3828f390df90583a5e5c1cc1f70870342b43a95c2d58f...,shipping_region,USAC/USGC,model,1
6,956b03529510459d298c4e537d23c6fbb22ffd8414af93...,trading_block,Non-OPEC,model,1
7,4e79eb8e84d26f5c3c0006283bc1aa52c170b58d667be9...,trading_region,PADD 3 (US Gulf Coast),model,1
8,3663536da3c85053556bc0d1a803481aa137a854da8bc8...,trading_subregion,Texas,model,1
9,3a39cf841ece0c7cb879f72af01cb634191142e0de8010...,terminal,Houston Enterprise Terminal,model,1


**A point to keep in mind**

In this lesson we have dived into the structure of a Cargo Movement. While important for understanding, in practice it is often simpler to just use cm_query.to_df('all') to convert all these records to a familiar tabular form, without paying attention to the structure. This is true of other endpoints as well.

### Exercise

The Vortexa SDK also offers a `VesselMovements` endpoint. Vessel Movements can be searched for in a similar way to Cargo Movements, but they have some differences. Do a query for Vessel Movements, and inspect the structure of a Vessel Movement to identify the differences.

[Vessel Movements documentation.](https://vortechsa.github.io/python-sdk/endpoints/vessel_movements/)